# MULTIPLY SAR Data Access and Pre-Processing

The purpose of this Jupyter Notebook is to show how the MULTIPLY platform can be used to retrieve S1 SLC Data from the Data Access Component and how it can be processed into S1 GRD Data using the SAR Pre-Processing functionality.

First, let's define working directories.

In [1]:
from vm_support import get_working_dir
name = 'm1'

# create and/or clear working directory
working_dir = get_working_dir(name)

In [2]:
print('Working directory is {}'.format(working_dir))
s1_slc_directory = '{}/s1_slc'.format(working_dir)
s1_grd_directory = '{}/s1_grd'.format(working_dir)

Working directory is /Data/t_fincke/m1


We need to define start and end times and a region of interest.

In [3]:
# start_time_as_string = '2018-05-10'
# stop_time_as_string = '2018-05-20'
# roi = "POLYGON((-2.20397502663252 39.09868106889479,-1.9142106223355313 39.09868106889479," \
#                  "-1.9142106223355313 38.94504502508093,-2.20397502663252 38.94504502508093," \
#                  "-2.20397502663252 39.09868106889479))"
start_time_as_string = '2017-06-01'
stop_time_as_string = '2017-06-10'
roi = 'POLYGON((9.8 53.6,10.2 53.6,10.2 53.4, 9.8 53.4, 9.8 53.6))'

For the SAR Pre-Processing we require a config file. Let's create it.

In [4]:
from vm_support import create_sar_config_file
create_sar_config_file(temp_dir=working_dir, roi=roi, start_time=start_time_as_string, end_time=stop_time_as_string,
                       s1_slc_directory=s1_slc_directory, s1_grd_directory=s1_grd_directory)
config_file = f'{working_dir}/sar_config.yaml'

Next set up the Data Access Component.

In [5]:
from multiply_data_access import DataAccessComponent
dac = DataAccessComponent()

INFO:root:Read data store aws_s2
INFO:root:Read data store cams
INFO:root:Read data store cams_tiff
INFO:root:Read data store emulators
INFO:root:Read data store wv_emulator
INFO:root:Read data store aster_dem
INFO:root:Read data store modis_mcd43a1
INFO:root:Read data store S2L2
INFO:root:Read data store MundiRest


For the SAR Pre-Processing we need to have 15 products at least: 7 before and 7 after the product in question. However, the SAR Pre-Processing does not count all products as full products: If the products are located close to a border, they are counted as half products. As the determination whether a product is counted as a full or half a product is made by the SAR Pre-Processor, we need it to determine he products that are required. To do so, it might also be necessary to download products.

Let's start with determining the actual start date:

In [6]:
import datetime
one_day = datetime.timedelta(days=1)
start = datetime.datetime.strptime(start_time_as_string, '%Y-%m-%d')
before = start
before -= one_day
before -= one_day
before -= one_day
before -= one_day
before -= one_day
before -= one_day
before -= one_day
before_date = datetime.datetime.strftime(before, '%Y-%m-%d')
data_urls_before = dac.get_data_urls(roi, before_date, start_time_as_string, 'S1_SLC')
# before_date
data_urls_before

['/Data/archive/S1_SLC/2017/05/25/S1A_IW_SLC__1SDV_20170525T170030_20170525T170057_016741_01BCE3_9B0A.zip',
 '/Data/archive/S1_SLC/2017/05/26/S1B_IW_SLC__1SDV_20170526T054019_20170526T054046_005765_00A1A5_5866.zip',
 '/Data/archive/S1_SLC/2017/05/26/S1B_IW_SLC__1SDV_20170526T054044_20170526T054111_005765_00A1A5_A40E.zip',
 '/Data/archive/S1_SLC/2017/05/27/S1A_IW_SLC__1SDV_20170527T053246_20170527T053314_016763_01BD9F_AFBF.zip',
 '/Data/archive/S1_SLC/2017/05/27/S1A_IW_SLC__1SDV_20170527T053311_20170527T053338_016763_01BD9F_C0E2.zip',
 '/Data/archive/S1_SLC/2017/05/30/S1A_IW_SLC__1SDV_20170530T170848_20170530T170915_016814_01BF29_4FBD.zip',
 '/Data/archive/S1_SLC/2017/05/31/S1B_IW_SLC__1SDV_20170531T165946_20170531T170013_005845_00A3F0_956C.zip',
 '/Data/archive/S1_SLC/2017/06/01/S1A_IW_SLC__1SDV_20170601T054058_20170601T054126_016836_01BFD8_0070.zip',
 '/Data/archive/S1_SLC/2017/06/01/S1A_IW_SLC__1SDV_20170601T054123_20170601T054150_016836_01BFD8_B122.zip']

In [7]:
import datetime
import logging
import os
from multiply_orchestration import create_sym_links
from sar_pre_processing import SARPreProcessor
one_day = datetime.timedelta(days=1)

before_sar_dir = f'{s1_slc_directory}/before'
if not os.path.exists(before_sar_dir):
    os.makedirs(before_sar_dir)

start = datetime.datetime.strptime(start_time_as_string, '%Y-%m-%d')
before = start
num_before = 0
while num_before < 7:
    before -= one_day
    before_date = datetime.datetime.strftime(before, '%Y-%m-%d')
    data_urls_before = dac.get_data_urls(roi, before_date, start_time_as_string, 'S1_SLC')
    create_sym_links(data_urls_before, before_sar_dir)
    processor = SARPreProcessor(config=config_file, input=before_sar_dir, output=before_sar_dir)
    list = processor.create_processing_file_list()
    num_before = len(list[0]) + (len(list[1]) / 2.)
logging.info(f'Set start date for collecting S1 SLC products to {before_date}.')

Number of found files for year 2017: 3
Number of found files containing area of interest: 3
Number of found files that were double processed: 0.0
Number of found files with border issues: 2
Number of found files for year 2017: 4
Number of found files containing area of interest: 4
Number of found files that were double processed: 0.0
Number of found files with border issues: 2
Number of found files for year 2017: 4
Number of found files containing area of interest: 4
Number of found files that were double processed: 0.0
Number of found files with border issues: 2
Number of found files for year 2017: 4
Number of found files containing area of interest: 4
Number of found files that were double processed: 0.0
Number of found files with border issues: 2
Number of found files for year 2017: 6
Number of found files containing area of interest: 6
Number of found files that were double processed: 0.0
Number of found files with border issues: 4
Number of found files for year 2017: 8
Number of f

INFO:root:Downloading S1B_IW_SLC__1SDV_20170524T170804_20170524T170831_005743_00A0F5_841A


99 %

INFO:root:Downloaded S1B_IW_SLC__1SDV_20170524T170804_20170524T170831_005743_00A0F5_841A


Number of found files for year 2017: 10


INFO:root:Set start date for collecting S1 SLC products to 2017-05-24.


Number of found files containing area of interest: 10
Number of found files that were double processed: 0.0
Number of found files with border issues: 6


Now the actual end date. Take care not to set it in the future.

In [9]:
after_sar_dir = f'{s1_slc_directory}/after'
if not os.path.exists(after_sar_dir):
    os.makedirs(after_sar_dir)

end = datetime.datetime.strptime(stop_time_as_string, '%Y-%m-%d')
after = end
num_after = 0
while num_after < 7 and after < datetime.datetime.today():
    after += one_day
    after_date = datetime.datetime.strftime(after, '%Y-%m-%d')
    data_urls_after = dac.get_data_urls(roi, stop_time_as_string, after_date, 'S1_SLC')
    create_sym_links(data_urls_after, after_sar_dir)
    processor = SARPreProcessor(config=config_file, input=after_sar_dir, output=after_sar_dir)
    list = processor.create_processing_file_list()
    num_after = len(list[0]) + (len(list[1]) / 2.)
logging.info(f'Set end date for collecting S1 SLC products to {after_date}.')

INFO:root:Downloading S1A_IW_SLC__1SDV_20170611T170848_20170611T170915_016989_01C495_4CDC


99 %

INFO:root:Downloaded S1A_IW_SLC__1SDV_20170611T170848_20170611T170915_016989_01C495_4CDC


Number of found files for year 2017: 1
Number of found files containing area of interest: 1
Number of found files that were double processed: 0.0
Number of found files with border issues: 0


INFO:root:Downloading S1B_IW_SLC__1SDV_20170612T165947_20170612T170014_006020_00A90D_02A0


...

99 %

INFO:root:Downloaded S1B_IW_SLC__1SDV_20170619T054034_20170619T054101_006115_00ABDC_1FDA


Number of found files for year 2017: 8


INFO:root:Set end date for collecting S1 SLC products to 2017-06-19.


Number of found files containing area of interest: 8
Number of found files that were double processed: 0.0
Number of found files with border issues: 2


We created extra directories for collecting the products. Let's clean up here.

In [10]:
import shutil

shutil.rmtree(before_sar_dir)
shutil.rmtree(after_sar_dir)

Now, we are finally set to collect the data:

In [11]:
sar_data_urls = dac.get_data_urls(roi, before_date, after_date, 'S1_SLC')
create_sym_links(sar_data_urls, s1_slc_directory)

INFO:root:Downloading S1B_IW_SLC__1SDV_20170602T053227_20170602T053254_005867_00A499_9917


99 %

INFO:root:Downloaded S1B_IW_SLC__1SDV_20170602T053227_20170602T053254_005867_00A499_9917
INFO:root:Downloading S1B_IW_SLC__1SDV_20170605T170805_20170605T170832_005918_00A60E_9818


99 %

INFO:root:Downloaded S1B_IW_SLC__1SDV_20170605T170805_20170605T170832_005918_00A60E_9818
INFO:root:Downloading S1A_IW_SLC__1SDV_20170606T170031_20170606T170058_016916_01C25A_5E28


...

INFO:root:Downloaded S1A_IW_SLC__1SDV_20170606T170031_20170606T170058_016916_01C25A_5E28
INFO:root:Downloading S1B_IW_SLC__1SDV_20170607T054033_20170607T054101_005940_00A6B7_41C0


99 %

INFO:root:Downloaded S1B_IW_SLC__1SDV_20170607T054033_20170607T054101_005940_00A6B7_41C0
INFO:root:Downloading S1A_IW_SLC__1SDV_20170608T053301_20170608T053328_016938_01C30F_C92D


99 %

INFO:root:Downloaded S1A_IW_SLC__1SDV_20170608T053301_20170608T053328_016938_01C30F_C92D


Now that the data has been collected, we can run the actual SAR Pre-Processing. The Processing consists of three steps. The first two steps create one output product for one input product, while the third step merges information from multiple products. We can run steps 1 and 2 safely now on all the input folders.

In [ ]:
processor = SARPreProcessor(config=config_file, input=s1_slc_directory, output=s1_grd_directory)
processor.create_processing_file_list()
logging.info('Start Pre-processing step 1')
processor.pre_process_step1()
logging.info('Finished Pre-processing step 1')
logging.info('Start Pre-processing step 2')
processor.pre_process_step2()
logging.info('Finished Pre-processing step 2')

Number of found files for year 2017: 23


INFO:root:Start Pre-processing step 1
INFO:root:normalisation angle not specified, default value of 35 is used for processing
INFO:root:Scene 1 of 15
INFO:root:Process S1A_IW_SLC__1SDV_20170525T170030_20170525T170057_016741_01BCE3_9B0A.zip with SNAP.


Number of found files containing area of interest: 23
Number of found files that were double processed: 0.0
Number of found files with border issues: 8


INFO:root:0
INFO:root:Scene 2 of 15
INFO:root:Process S1A_IW_SLC__1SDV_20170530T170848_20170530T170915_016814_01BF29_4FBD.zip with SNAP.


Step 3 needs to be performed for each product separately. To do this, we need to make sure we hand in the correct products only. The output of the second step is located in an intermediate folder. First, we collect all these files and sort them temporally.

In [ ]:
output_step2_dir = f'{s1_grd_dir}/step2'
input_files = glob.glob(f'{output_step2_dir}/*.dim')
input_file_dict = {}
for input_file in input_files:
    input_file_dict[input_file[-68:]] = input_file
sorted_input_file_dict = sorted(input_file_dict)
sorted_input_files = []
for sorted_input_file in sorted_input_file_dict:
    sorted_input_files.append(input_file_dict[sorted_input_file])

Now we can run the thrird step of the SAR Pre-Processing for every product for which there are at least 7 products before and 7 products after it available. For this, it is necessary to first create the file list, then to remove all files from it that shall not be considered during this step.

In [ ]:
for end in range(14, len(input_files)):
    file_list = processor.create_processing_file_list()
    start = end-14
    sub_list = sorted_input_files[start:end]
    for i, list in enumerate(file_list):
        for file in list:
            processed_name = file.replace('.zip', '_GC_RC_No_Su_Co.dim')
            processed_name = processed_name.replace(s1_slc_directory, output_step2_dir)
            if processed_name not in sub_list:
                list.remove(file)
                logging.info(f'Removing file {file}')
    processor.set_file_list(file_list)
    logging.info(f'Start Pre-processing step 3, run {start}')
    processor.pre_process_step3()
    logging.info(f'Finished Pre-processing step 3, run {start}')
    files = os.listdir(output_step3_dir)
    for file in files:
        shutil.move(os.path.join(output_step3_dir, file), output_dir)